# Генеративные сети

Рекуррентные нейронные сети (RNN) и их варианты с управляемыми ячейками, такие как ячейки долгой краткосрочной памяти (LSTM) и управляемые рекуррентные блоки (GRU), предоставляют механизм для языкового моделирования, то есть они могут обучаться порядку слов и предсказывать следующее слово в последовательности. Это позволяет использовать RNN для **генеративных задач**, таких как обычная генерация текста, машинный перевод и даже создание подписей к изображениям.

В архитектуре RNN, которую мы обсуждали в предыдущем разделе, каждая RNN-ячейка выдавала следующее скрытое состояние в качестве результата. Однако мы также можем добавить еще один выход к каждой рекуррентной ячейке, что позволит нам выдавать **последовательность** (равную по длине исходной последовательности). Более того, мы можем использовать RNN-ячейки, которые не принимают входные данные на каждом шаге, а просто используют некоторый начальный вектор состояния и затем генерируют последовательность выходных данных.

В этом ноутбуке мы сосредоточимся на простых генеративных моделях, которые помогают нам генерировать текст. Для упрощения давайте создадим **символьную сеть**, которая генерирует текст по буквам. Во время обучения нам нужно взять некоторый текстовый корпус и разбить его на последовательности букв.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Создание словаря символов

Чтобы построить генеративную сеть на уровне символов, необходимо разбить текст на отдельные символы, а не на слова. Слой `TextVectorization`, который мы использовали ранее, не может выполнить эту задачу, поэтому у нас есть два варианта:

* Загрузить текст вручную и выполнить токенизацию "вручную", как показано в [этом официальном примере Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Использовать класс `Tokenizer` для токенизации на уровне символов.

Мы выберем второй вариант. `Tokenizer` также можно использовать для токенизации на уровне слов, поэтому переключение между токенизацией символов и слов должно быть достаточно простым.

Для токенизации на уровне символов необходимо передать параметр `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Мы также хотим использовать один специальный токен для обозначения **конца последовательности**, который мы назовем `<eos>`. Давайте добавим его вручную в словарь:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Обучение генеративной RNN для создания заголовков

Метод, который мы будем использовать для обучения RNN создавать заголовки новостей, следующий. На каждом шаге мы берем один заголовок, который подается в RNN, и для каждого входного символа сеть должна сгенерировать следующий выходной символ:

![Изображение, показывающее пример генерации слова 'HELLO' с помощью RNN.](../../../../../translated_images/ru/rnn-generate.56c54afb52f9781d.webp)

Для последнего символа нашей последовательности мы попросим сеть сгенерировать токен `<eos>`.

Основное отличие генеративной RNN, которую мы используем здесь, заключается в том, что мы будем брать выходные данные с каждого шага RNN, а не только из последней ячейки. Это можно сделать, указав параметр `return_sequences` для ячейки RNN.

Таким образом, во время обучения входом для сети будет последовательность закодированных символов определенной длины, а выходом — последовательность той же длины, но сдвинутая на один элемент и завершенная токеном `<eos>`. Минибатч будет состоять из нескольких таких последовательностей, и нам потребуется использовать **дополнение** (padding), чтобы выровнять все последовательности.

Давайте создадим функции, которые будут преобразовывать набор данных для нас. Поскольку мы хотим дополнять последовательности на уровне минибатча, сначала мы сгруппируем набор данных, вызвав `.batch()`, а затем применим `map`, чтобы выполнить преобразование. Таким образом, функция преобразования будет принимать целый минибатч в качестве параметра:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Несколько важных моментов, которые мы делаем здесь:
* Сначала мы извлекаем фактический текст из строкового тензора
* `text_to_sequences` преобразует список строк в список целочисленных тензоров
* `pad_sequences` дополняет эти тензоры до их максимальной длины
* В конце мы выполняем one-hot кодирование всех символов, а также сдвиг и добавление `<eos>`. Скоро мы увидим, почему нам нужны символы в формате one-hot

Однако эта функция является **Pythonic**, то есть она не может быть автоматически преобразована в вычислительный граф Tensorflow. Если мы попытаемся использовать эту функцию напрямую в `Dataset.map`, возникнут ошибки. Нам нужно обернуть этот вызов с помощью оболочки `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Различие между функциями преобразования в стиле Python и Tensorflow может показаться слишком сложным, и вы можете задаться вопросом, почему мы не преобразуем набор данных с помощью стандартных функций Python перед передачей его в `fit`. Хотя это действительно возможно, использование `Dataset.map` имеет огромное преимущество, так как конвейер преобразования данных выполняется с использованием вычислительного графа Tensorflow, который использует преимущества вычислений на GPU и минимизирует необходимость передачи данных между CPU и GPU.

Теперь мы можем построить нашу генеративную сеть и начать обучение. Она может быть основана на любой рекуррентной ячейке, которую мы обсуждали в предыдущем разделе (простой, LSTM или GRU). В нашем примере мы будем использовать LSTM.

Поскольку сеть принимает символы в качестве входных данных, а размер словаря довольно мал, нам не нужен слой embedding — входные данные в формате one-hot могут напрямую подаваться в ячейку LSTM. Выходной слой будет классификатором `Dense`, который преобразует выход LSTM в номера токенов в формате one-hot.

Кроме того, так как мы работаем с последовательностями переменной длины, мы можем использовать слой `Masking`, чтобы создать маску, которая будет игнорировать дополненные части строки. Это не является строго необходимым, так как нас не слишком интересует всё, что идёт после токена `<eos>`, но мы будем использовать этот слой для получения опыта работы с ним. `input_shape` будет иметь вид `(None, vocab_size)`, где `None` указывает на последовательность переменной длины, а выходная форма также будет `(None, vocab_size)`, как вы можете увидеть из `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Генерация вывода

Теперь, когда мы обучили модель, мы хотим использовать её для генерации вывода. Прежде всего, нам нужен способ декодировать текст, представленный последовательностью чисел токенов. Для этого мы могли бы использовать функцию `tokenizer.sequences_to_texts`; однако она плохо работает с токенизацией на уровне символов. Поэтому мы возьмём словарь токенов из токенизатора (называемый `word_index`), создадим обратную карту и напишем собственную функцию декодирования:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Теперь начнем генерацию. Мы начнем с некоторой строки `start`, закодируем ее в последовательность `inp`, а затем на каждом шаге будем вызывать нашу сеть для предсказания следующего символа.

Выход сети `out` — это вектор из `vocab_size` элементов, представляющий вероятности каждого токена, и мы можем найти наиболее вероятный номер токена, используя `argmax`. Затем мы добавляем этот символ в сгенерированный список токенов и продолжаем генерацию. Этот процесс генерации одного символа повторяется `size` раз для создания необходимого количества символов, и мы завершаем процесс досрочно, если встречается `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Выборка результатов во время обучения

Поскольку у нас нет полезных метрик, таких как *точность*, единственный способ увидеть, что наша модель становится лучше — это **выборка** сгенерированных строк во время обучения. Для этого мы будем использовать **колбэки**, то есть функции, которые мы можем передать в функцию `fit`, и которые будут вызываться периодически в процессе обучения.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Этот пример уже генерирует довольно хороший текст, но его можно улучшить несколькими способами:
* **Больше текста**. Мы использовали только заголовки для нашей задачи, но вы можете попробовать работать с полным текстом. Помните, что RNN не слишком хорошо справляются с обработкой длинных последовательностей, поэтому имеет смысл либо разбивать их на более короткие предложения, либо всегда обучать на фиксированной длине последовательности с заранее заданным значением `num_chars` (например, 256). Вы можете попробовать изменить пример выше на такую архитектуру, используя [официальный учебник Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) в качестве вдохновения.
* **Многослойный LSTM**. Имеет смысл попробовать 2 или 3 слоя ячеек LSTM. Как мы упоминали в предыдущем разделе, каждый слой LSTM извлекает определенные шаблоны из текста, и в случае генератора на уровне символов можно ожидать, что нижний уровень LSTM будет отвечать за извлечение слогов, а более высокие уровни — за слова и их комбинации. Это можно легко реализовать, передав параметр количества слоев в конструктор LSTM.
* Вы также можете попробовать экспериментировать с **GRU-ячейками** и посмотреть, какие из них работают лучше, а также с **различными размерами скрытых слоев**. Слишком большой скрытый слой может привести к переобучению (например, сеть будет запоминать точный текст), а меньший размер может не дать хорошего результата.


## Генерация мягкого текста и температура

В предыдущем определении функции `generate` мы всегда выбирали символ с наибольшей вероятностью в качестве следующего символа в генерируемом тексте. Это приводило к тому, что текст часто "зацикливался" на одних и тех же последовательностях символов снова и снова, как в этом примере:
```
today of the second the company and a second the company ...
```

Однако, если мы посмотрим на распределение вероятностей для следующего символа, может оказаться, что разница между несколькими наивысшими вероятностями незначительна, например, один символ может иметь вероятность 0.2, а другой — 0.19 и т.д. Например, при поиске следующего символа в последовательности '*play*', следующим символом может быть как пробел, так и **e** (как в слове *player*).

Это приводит нас к выводу, что не всегда "справедливо" выбирать символ с наивысшей вероятностью, так как выбор второго по величине вероятности также может привести к осмысленному тексту. Более разумным подходом будет **выбор** символов из распределения вероятностей, предоставленного выходом сети.

Этот выбор можно осуществить с помощью функции `np.multinomial`, которая реализует так называемое **многомерное распределение**. Функция, реализующая эту **мягкую** генерацию текста, определена ниже:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Мы ввели еще один параметр, называемый **temperature**, который используется для указания того, насколько строго мы должны придерживаться наивысшей вероятности. Если temperature равно 1.0, мы выполняем честное мультиномиальное выборочное моделирование, а когда temperature стремится к бесконечности, все вероятности становятся равными, и мы случайным образом выбираем следующий символ. В приведенном ниже примере можно наблюдать, что текст становится бессмысленным, если слишком сильно увеличить значение temperature, и он напоминает "циклически" жестко сгенерированный текст, когда значение приближается к 0.



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Несмотря на наши усилия обеспечить точность, автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
